<a href="https://colab.research.google.com/github/haakonnese/tdt-4173-revenue/blob/main/Auto_sklearn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!sudo apt-get install build-essential swig
!pip install auto-sklearn==0.15.0

Reading package lists... Done
Building dependency tree       
Reading state information... Done
build-essential is already the newest version (12.4ubuntu1).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
Suggested packages:
  swig-doc swig-examples swig3.0-examples swig3.0-doc
The following NEW packages will be installed:
  swig swig3.0
0 upgraded, 2 newly installed, 0 to remove and 27 not upgraded.
Need to get 1,100 kB of archives.
After this operation, 5,822 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 swig3.0 amd64 3.0.12-1 [1,094 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 swig amd64 3.0.12-1 [6,460 B]
Fetched 1,100 kB in 1s (1,253 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/Fron

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, KFold
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder, LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_squared_log_error, make_scorer
from sklearn.feature_selection import SelectKBest, chi2, SelectFromModel, mutual_info_classif
from sklearn.impute import SimpleImputer
from sklearn.svm import LinearSVR
import autosklearn.regression

In [ ]:
def rmsle(y_true, y_pred):
    """
    Computes the Root Mean Squared Logarithmic Error 
    
    Args:
        y_true (np.array): n-dimensional vector of ground-truth values 
        y_pred (np.array): n-dimensional vecotr of predicted values 
    
    Returns:
        A scalar float with the rmsle value 
    
    Note: You can alternatively use sklearn and just do: 
        `sklearn.metrics.mean_squared_log_error(y_true, y_pred) ** 0.5`
    """
    y_pred[y_pred < 0] = 0
    assert (y_true >= 0).all(), 'Received negative y_true values'
    assert (y_pred >= 0).all(), 'Received negative y_pred values'
    assert y_true.shape == y_pred.shape, 'y_true and y_pred have different shapes'
    y_true_log1p = np.log1p(y_true)  # log(1 + y_true)
    y_pred_log1p = np.log1p(y_pred)  # log(1 + y_pred)
    return np.sqrt(np.mean(np.square(y_pred_log1p - y_true_log1p)))

In [ ]:
df = pd.read_csv("all_with_stores_pop").set_index(["dataset", "range_index"])
#df["latlon"] = np.round(df["lat"], decimals = 4).astype(str) + "-" + np.round(df["lon"], decimals = 4).astype(str)
df.drop(columns=['store_name', 'address', 'lat', 'lon', 'busstop_id', 'importance_level', 'stopplace_type', 'grunnkrets_id'], inplace=True)
df['in_mall'] = df['mall_name'].notna()
df['in_chain'] = df['chain_name'].notna()
# df['stopplace_type'] = df['stopplace_type'].fillna("Mangler type")
df['mall_name'] = df['mall_name'].fillna("None")
#df['address'] = df['address'].fillna("None")
#df['stopplace_type'] = df['stopplace_type'].fillna("None")

df['chain_name'] = df['chain_name'].fillna("None")
# df['busstop_id'] = df['busstop_id'].map(str)
df['lv1'] = df['lv1'].map(str)
df['lv2'] = df['lv2'].map(str)
df['lv3'] = df['lv3'].map(str)
df['lv4'] = df['lv4'].map(str)

data_with_label = df.loc["train"]
data_with_label.set_index('store_id', inplace=True)

data_with_label = data_with_label[data_with_label.revenue < 80]

X, y = data_with_label.loc[:, data_with_label.columns != 'revenue'], data_with_label['revenue']

# Numeric feature selection
# print(X.shape)
# numeric_features = X.select_dtypes(include=[np.number]).columns
# categorical_features = X.select_dtypes(include=[np.object0]).columns
# X = X[categorical_features].join(X[['singles_y', 'grunnkrets_population', 'municipality_density',
#  'district_density', 'lv1_population_district_div_count_stores',
#  'lv3_population_district_div_count_stores',
#  'lv4_population_district_div_count_stores',
#  'lv1_population_municipality_div_count_stores',
#  'lv2_population_municipality_div_count_stores',
#  'lv3_population_municipality_div_count_stores']])

# Combining categories
# for cat_name in categorical_features:
#     series = X[cat_name].value_counts()
#     mask = (series/series.sum() * 100).lt(.05)
#     # To replace df['column'] use np.where I.e 
#     X[cat_name] = np.where(X[cat_name].isin(series[mask].index),'Other',X[cat_name])


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=43)
y_train_log = np.log1p(y_train)
y_log = np.log1p(y)

In [ ]:
X.shape

(12698, 58)

In [ ]:
numeric_features = X.select_dtypes(include=[np.number]).columns
numeric_transformer = Pipeline(
    steps=[("imputer", SimpleImputer()),
        ("scaler", StandardScaler())]
)

categorical_features = X.select_dtypes(include=[np.object0]).columns
categorical_transformer = OneHotEncoder(handle_unknown="ignore")

preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features),
    ],
    remainder="passthrough"
)

In [ ]:
def error(y_true, y_pred):
    y_pred[y_pred < 0] = 0
    y_pred = np.expm1(y_pred)
    return np.sqrt(mean_squared_log_error(y_true, y_pred))

In [ ]:
TIME = 4000

automl = autosklearn.regression.AutoSklearnRegressor(
  time_left_for_this_task=TIME,
  per_run_time_limit=TIME/10,
  memory_limit=2*10**12+1,
  n_jobs=-1,
  resampling_strategy='cv',
  resampling_strategy_arguments={'folds': 5}
)

pipeline = Pipeline(
    steps=[("preprocessor", preprocessor), 
    ("model", 
    automl)])
pipeline.fit(
  X,
  y_log
)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  Index(['other_stores_1000', 'other_stores_100', 'other_stores_50',
       'other_stores_250', 'buss_stops_1000', 'buss_stops_300', 'grunnkrets_1',
       'distance_closest_busstop', 'area_km2', 'couple_ch...
                                                  Index(['chain_name', 'mall_name', 'district_name', 'municipality_name', 'lv1',
       'lv2', 'lv3', 'lv4', 'side_placement'],
      dtype='object'))])),
                ('model',
                 AutoSklearnRegress

In [ ]:
preds = pipeline.predict(X)
print(error(y, preds))

0.6503060185853666


In [ ]:
# This leaderboard is from a run of 12000s with only standard preprocessing
pipeline[1].leaderboard(detailed = True, ensemble_only=False)

,rank,ensemble_weight,type,cost,duration,config_id,train_loss,seed,start_time,end_time,budget,status,data_preprocessors,feature_preprocessors,balancing_strategy,config_origin
model_id,,,,,,,,,,,,,,,,
13,1,0.30,libsvm_svr,5.179335e-01,852.296654,12,4.318307e-01,0,1.666533e+09,1.666533e+09,0.0,StatusType.SUCCESS,[],[no_preprocessing],None,Initial design
29,2,0.10,libsvm_svr,5.188854e-01,63.841393,28,4.638629e-01,0,1.666537e+09,1.666537e+09,0.0,StatusType.SUCCESS,[],[select_percentile_regression],None,Random Search (sorted)
27,3,0.00,liblinear_svr,5.220834e-01,13.755913,26,4.988931e-01,0,1.666537e+09,1.666537e+09,0.0,StatusType.SUCCESS,[],[select_rates_regression],None,Random Search (sorted)
24,4,0.20,random_forest,5.232402e-01,1002.342208,23,3.473289e-01,0,1.666535e+09,1.666536e+09,0.0,StatusType.SUCCESS,[],[select_percentile_regression],None,Initial design
47,5,0.00,liblinear_svr,5.278974e-01,12.630491,46,5.137630e-01,0,1.666540e+09,1.666540e+09,0.0,StatusType.SUCCESS,[],[select_rates_regression],None,Local Search
71,6,0.00,sgd,5.294418e-01,11.268063,70,5.139919e-01,0,1.666542e+09,1.666542e+09,0.0,StatusType.SUCCESS,[],[select_rates_regression],None,Local Search
3,7,0.10,liblinear_svr,5.313211e-01,8.363142,2,4.457350e-01,0,1.666531e+09,1.666531e+09,0.0,StatusType.SUCCESS,[],[select_percentile_regression],None,Initial design
23,8,0.12,random_forest,5.316083e-01,733.558452,22,3.063630e-01,0,1.666535e+09,1.666536e+09,0.0,StatusType.SUCCESS,[],[select_rates_regression],None,Initial design
76,9,0.00,sgd,5.359885e-01,11.278381,75,5.248102e-01,0,1.666542e+09,1.666542e+09,0.0,StatusType.SUCCESS,[],[select_rates_regression],None,Local Search


In [ ]:
pipeline[1].sprint_statistics()

'auto-sklearn results:\n  Dataset name: ffd806f4-504d-11ed-803e-0242ac1c0002\n  Metric: r2\n  Best validation score: 0.450286\n  Number of target algorithm runs: 27\n  Number of successful target algorithm runs: 4\n  Number of crashed target algorithm runs: 6\n  Number of target algorithms that exceeded the time limit: 17\n  Number of target algorithms that exceeded the memory limit: 0\n'

In [ ]:
test = df.loc["test"]

In [ ]:
test.drop(columns="revenue", inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [ ]:
test.set_index('store_id', inplace=True)

In [ ]:
preds_sub = pipeline.predict(test)
print(len(preds_sub))
preds_sub = np.expm1(preds_sub)

8577


In [ ]:
preds_sub

array([5.35774955, 5.30620134, 2.28087197, ..., 4.92194701, 5.58106576,
       1.63262937])

In [ ]:
len(test.index)

8577

In [ ]:
sub = pd.Series(name="predicted",data=preds_sub, index=test.index)
sub.index.name = "id"
sub

id
884875072-975988988-41051     5.357750
999235174-999504078-507807    5.306201
916715463-816878292-826266    2.280872
999086284-982563895-99598     3.105735
979879741-972291455-25435     3.745821
                                ...   
917064016-817211402-838350    2.294350
917353379-917411824-845904    4.961387
917323003-917383529-844309    4.921947
937628668-972044857-47364     5.581066
952390821-971994908-26248     1.632629
Name: predicted, Length: 8577, dtype: float64